In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!git clone https://github.com/bstri/cs616

Cloning into 'cs616'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.


In [0]:
#import shutil
#shutil.rmtree('eclipse.platform.ui')

!git clone https://github.com/eclipse/eclipse.platform.ui


/bin/bash: import: command not found
/bin/bash: -c: line 0: syntax error near unexpected token `'eclipse.platform.ui''
/bin/bash: -c: line 0: `shutil.rmtree('eclipse.platform.ui')'
Cloning into 'eclipse.platform.ui'...
remote: Enumerating objects: 2532, done.
remote: Counting objects: 100% (2532/2532), done.
remote: Compressing objects: 100% (1288/1288), done.
remote: Total 562625 (delta 1061), reused 1754 (delta 584), pack-reused 560093
Receiving objects: 100% (562625/562625), 119.59 MiB | 15.80 MiB/s, done.
Resolving deltas: 100% (320233/320233), done.


In [0]:
!pip install pydriller
!pip install ciso8601

     |████████████████████████████████| 71kB 2.4MB/s 
     |████████████████████████████████| 460kB 8.2MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
  Created wheel for ciso8601: filename=ciso8601-2.1.3-cp36-cp36m-linux_x86_64.whl size=28299 sha256=7dc0e956c7eed7140a76bf23627d7557a9d1d62dd3218015922e4b50a4929a3a
  Stored in directory: /root/.cache/pip/wheels/eb/32/e4/13bdaf7e245f82667b21e0cfb03d21224691a47fa9f9bc80a6
Successfully built ciso8601


Import and parse bug report info into Pandas DataFrame

In [0]:
import pandas as pd
#import xml.etree.ElementTree as ET

# This is a one-liner for the record books
# brdf = pd.DataFrame([{c.get('name'): ' '.join((c.text or '').split()) for c in el} for el in ET.parse('eclipseBugReports.xml').getroot()[1]])

brdf = pd.read_csv('cs616/updated_brdf.csv')

In [0]:
brdf.describe()

,id,bug_id,report_timestamp,commit_timestamp,previous_commit_timestamp
count,6495.000000,6495.000000,6.495000e+03,6.495000e+03,6.495000e+03
mean,3248.000000,177200.583526,1.170273e+09,1.178887e+09,1.178849e+09
std,1875.089331,117281.544559,9.527768e+07,9.534308e+07,9.531107e+07
min,1.000000,2025.000000,1.002753e+09,1.002824e+09,1.002819e+09
25%,1624.500000,70998.500000,1.091019e+09,1.103106e+09,1.103099e+09
50%,3248.000000,165690.000000,1.164287e+09,1.174842e+09,1.174840e+09
75%,4871.500000,283442.500000,1.247579e+09,1.259523e+09,1.259523e+09
max,6495.000000,424252.000000,1.387282e+09,1.389886e+09,1.389962e+09


Initialize eclipse UI repository mining object and eclipse UI repo



In [0]:
import pydriller
import datetime

eclipse_repo_path = "eclipse.platform.ui"

# Limit included commits to the specified time frame
eclipse_repo_mine = \
    pydriller.RepositoryMining(eclipse_repo_path,
                               since=datetime.datetime(2001, 10, 10, 0, 0, 0),
                               to=datetime.datetime(2014, 1, 17, 23, 59, 59))

eclipse_repo = pydriller.GitRepository(eclipse_repo_path)

# New Section

Note: code commented out because we are now reading the modified data
from file.

Change truncated `commit` hash to full hash, change rounded `commit_timestamp` to full timestamp, and make `report_timestamp` more exact based on `report_time`.

In [0]:
# import ciso8601

# Note: Some of these changes may not be strictly necessary, as it turns out
# that the functions for listing files as they existed at a commit and
# extracting source files will accept a truncated hash. Nevertheless, having
# more exact times is useful for finding the commit that was current for each
# bug report.

# commits_affected = 0
# sheet_row = brdf.shape[0] - 1
# for c in eclipse_repo_mine.traverse_commits():
#     trunc_hash = c.hash[0:7]
#     if trunc_hash == brdf.commit[sheet_row][0:7]:
#         brdf.commit[sheet_row] = str(c.hash)
#         brdf.commit_timestamp[sheet_row] = int(c.author_date.timestamp())

        # I learned this method of converting a formatted date string to a
        # timestamp from https://stackoverflow.com/a/27523634
        # brdf.report_timestamp[sheet_row] = \
        #     int(ciso8601.parse_datetime(brdf.report_time[sheet_row]).timestamp())

        # commits_affected += 1
        # if (commits_affected == len(brdf.commit.unique())):
        #     break
        # sheet_row -= 1

In [0]:
brdf.head()

,id,bug_id,summary,description,report_time,report_timestamp,status,commit,commit_timestamp,previous_commit,previous_commit_timestamp,files,result
0,1,384108,Bug 384108 JUnit view icon no longer shows pro...,Build Identifier: Version: Juno Release Build ...,2012-07-03 03:39:25,1341286765,resolved fixed,5da5952ac820ee036bf94b225389c550c9fd6e35,1389711510,837a195cd9d56e9b8bba03fee6f4e80db753b6d1,1389962476,bundles/org.eclipse.e4.ui.workbench/src/org/ec...,123:bundles/org.eclipse.e4.ui.workbench/src/or...
1,2,385394,Bug 385394 Performance issue regarding enabled...,Build Identifier: Version: 4.2.0 Build id: I20...,2012-07-18 07:41:06,1342597266,resolved fixed,8db6c323e9849920d98e722c437a1f531b97cbdd,1389885883,6cc7cccd973320f28579a4446220d708487809b6,1389887401,bundles/org.eclipse.e4.ui.workbench.renderers....,6:bundles/org.eclipse.e4.ui.workbench.renderer...
2,3,423588,"Bug 423588 [QuickAccess] Quick Access failure,...",I was working in an inner from master. When I ...,2013-12-09 08:48:44,1386578924,resolved fixed,6750d2546700063bebc6a9567132e7c3f2671e3c,1389816531,602d549f4d027720a55a1983150f5610421d4bcd,1389803076,bundles/org.eclipse.ui.workbench/Eclipse UI/or...,81:bundles/org.eclipse.ui.workbench/Eclipse UI...
3,4,420238,Bug 420238 [CSS] Reduce whitespace usage in th...,One of the major complains I hear is that the ...,2013-10-24 04:08:35,1382587715,verified fixed,77108408a6915617564b031709f0596fa7c75394,1389800398,1f2e7c9d79cc5f66076df1daa952acd0bf96e161,1389799698,bundles/org.eclipse.e4.ui.workbench.renderers....,270:bundles/org.eclipse.e4.ui.workbench.render...
4,5,405216,Bug 405216 Widget disposed exception in Progre...,I have a job that sets the IProgressConstants....,2013-04-08 18:21:10,1365445270,resolved fixed,470f2747e7f4f598aa34b2b109a0bd833dbab942,1387385065,828d765394e7a0af8fca85bc324ad0afcde295ad,1389723936,bundles/org.eclipse.ui.workbench/Eclipse UI/or...,1:bundles/org.eclipse.ui.workbench/Eclipse UI/...


In [0]:
brdf.tail()

,id,bug_id,summary,description,report_time,report_timestamp,status,commit,commit_timestamp,previous_commit,previous_commit_timestamp,files,result
6490,6491,5721,Bug 5721 Welcome not disabled,NaN,2001-11-09 14:37:38,1005316658,resolved fixed,16eec34d596d51bb250532ce730c2c12f568c93d,1005337452,9fdcf1469f8729111aa07c9cc27124790b6db6f7,1005336876,bundles/org.eclipse.ui/Eclipse UI/org/eclipse/...,NaN
6491,6492,2694,Bug 2694 Editor Selection Dialog: UI suggestio...,MA (09.08.2001 12:04:06) 1. in the preference ...,2001-10-10 22:41:40,1002753700,resolved fixed,74a0be933573348e798737a9b4a2347208d2c421,1003251798,4dd7c7bf8be4f55264f5ece422a1555bcf38ec9c,1003249922,bundles/org.eclipse.ui/Eclipse UI/org/eclipse/...,NaN
6492,6493,2682,Bug 2682 [UI] EC: Can't contribute submenu ite...,"EC post ""Menus and submenus"" by GB on Aug 2, 2...",2001-10-10 22:41:23,1002753683,resolved fixed,cec14bf0fc360f8a4877a16e446414a897c1cd22,1003162748,7f3eada09c69c9ad37d91cfcf08941d35b58d2f9,1002918537,bundles/org.eclipse.ui/Eclipse JFace/org/eclip...,NaN
6493,6494,2558,Bug 2558 [UI] Unzoom can cause view to move ta...,- Open the resource perspective and reset it -...,2001-10-10 22:38:45,1002753525,resolved fixed,254977b62342d954bc5ea92895d7867a27f266df,1002917813,d2517d30726892c9b4795d44148d134c162e82eb,1002913558,bundles/org.eclipse.ui/Eclipse UI/org/eclipse/...,NaN
6494,6495,2318,Bug 2318 [UI] Type Hierarchy View resize (1GEN...,jkca (6/1/2001 5:44:46 PM) jre-sdk 116 To dupl...,2001-10-10 22:33:57,1002753237,resolved fixed,8fe046d61ffe811705f28871ea56ed7bb69415e9,1002824440,4953e3e2db29253276e54381f4535c8c245bba20,1002819197,bundles/org.eclipse.ui/Eclipse UI/org/eclipse/...,NaN


Note: code commented out because we are now reading the modified data from file.

Update brdf with hashes and timestamps of the commits that were current at the time of each bug report, and the hashes and timestamps of the commit preceding each bug fix commit.

In [0]:
# import dateutil

# commits_at_reports       = pd.Series([], dtype="string")
# commits_at_reports_times = pd.Series([], dtype="int")


# Get the commit that was current at the time of each bug report and add its
# hash and timestamp to brdf.
# for i, rt in enumerate(brdf['report_timestamp']):
#     c = None
#     rtdt = datetime.datetime.fromtimestamp(rt)

#     e_repo_mine_small = \
#         pydriller.RepositoryMining(eclipse_repo_path,
#                                    since=datetime.datetime(2001, 10, 10, 0, 0,
#                                                            0),
#                                    to=rtdt, order='reverse')

    # Since we have specified "order='reverse'", the first commit traversed is
    # usually the one immediately prior to the bug report, unless they were
    # submitted at nearly the exact same time. However we still check to ensure
    # that the chosen commit was before the bug report.
    # for c in e_repo_mine_small.traverse_commits():
    #     if datetime.datetime.fromtimestamp(c.author_date.timestamp()) < rtdt:
    #         commits_at_reports[i]       = str(c.hash)
    #         commits_at_reports_times[i] = int(c.author_date.timestamp())
    #         break
    #     else:
            # Debugging
#             print(f'Commit "{c.hash}" ({c.author_date.timestamp()}) not before '
#                   f'bug report at index {i} ({rt}).')

# brdf.insert(6, 'commit_at_report', commits_at_reports)
# brdf.insert(7, 'commit_at_report_timestamp', commits_at_reports_times)


# prev_commits       = pd.Series([], dtype="string")
# prev_commits_times = pd.Series([], dtype="int")

# Get the commit that immediately preceded the bug fix commit and add its hash
# and timestamp to brdf.
# for i, commit_hash in enumerate(brdf['commit']):
    # Modified from the checkout command included in the original project
    # README.pdf.
    # SO answers https://stackoverflow.com/a/41716996,
    # https://stackoverflow.com/a/19812608, and
    # https://stackoverflow.com/a/1828259 were helpful for getting the
    # formatting right.
#     prev_commit_hash, prev_commit_ts = \
#         eclipse_repo.git.execute(['git', 'show', '--pretty=format:"%H %ct',
#                                   '--quiet', f'{commit_hash}~1']).split()
#     prev_commit_hash      = prev_commit_hash.lstrip('"')
#     prev_commit_ts        = int(prev_commit_ts)
#     prev_commits[i]       = prev_commit_hash
#     prev_commits_times[i] = prev_commit_ts

# brdf.insert(9, 'previous_commit', prev_commits)
# brdf.insert(10, 'previous_commit_timestamp', prev_commits_times)

# brdf.head()

In [0]:
# Saving to csv so we have the option of simply reading the updated data from
# file.
# brdf.to_csv('updated_brdf.csv', encoding='utf8', index=False)

Define function to collect source code from repo given file name and commit, a function that will list the entire file tree at a particular commit, and a function that will return all source code a commit's file tree.

In [0]:
import git
import pathlib

recent_commit = 'a84b0f91988cc76f11094b507cdb4832a41502db'

def extractSrc(filepath, commit_hash):
    # Returns the text of `filepath` in `commit_hash` as a string
    src = None
    try:
        src = eclipse_repo.git.show(f'{commit_hash}:{filepath}')
    except git.GitCommandError:
        print('err')
        pass
    return src


def ls_commit(commit_hash):
    # List all source files in commit. Trying with reset because show was too
    # slow and inaccurate.
    tree = None
    eclipse_repo.git.reset('--hard', commit_hash)
    eclipse_repo.git.clean('-xdf')
    ecp = pathlib.Path(eclipse_repo_path)
    tree = [str(fp) for fp in ecp.rglob('*') if fp.suffix == '.java'
            and not fp.name.startswith('.')]
    # switch back
    eclipse_repo.git.reset('--hard', recent_commit)
    eclipse_repo.git.clean('-xdf')
    return tuple(tree)


def extract_commit_corpus(commit_hash, file_list=None, return_names=False):
    # Returns a tuple of strings, each of which represents one document in the
    # repo. If given a file_list, only the files in the file list are returned
    corpus  = []
    tree = None
    eclipse_repo.git.reset('--hard', commit_hash)
    eclipse_repo.git.clean('-xdf')
    ecp = pathlib.Path(eclipse_repo_path)
    # get only java or class files and no hidden files
    tree = sorted([str(fp) for fp in ecp.rglob('*') if fp.suffix == '.java'
                   and not fp.name.startswith('.')])
    if file_list:
        mod_tree = sorted([fp for fp in tree if re.sub(f'{eclipse_repo_path}/',
                                                       '', fp) in file_list])
        tree = mod_tree
    for doc in tree:
        src = None
        # Read only ascii characters, replace others with '?'. This is necessary
        # because some files are apparently not unicode.
        with open(doc, 'r', encoding='ascii', errors='replace') as f:
            src = f.read()
        if src is not None:
            if return_names:
                corpus.append(re.sub(f'{eclipse_repo_path}/', '', doc))
            corpus.append(src)
    # switch back
    eclipse_repo.git.reset('--hard', recent_commit)
    eclipse_repo.git.clean('-xdf')
    return tuple(corpus)

Define functions for preparing dataset

In [0]:
import math
import numpy
import re
import string
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import porter, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer


def split_camelcase(word):
    if word.isupper() or word.islower():
        split_words = [word]
    else:
        # One liner from https://stackoverflow.com/a/37697078
        # Inserts a blank before each capital letter, and then splits the
        # resulting string using the blanks.
        split_words = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1',
                                                             word)).split()
    return split_words


def preprocess(txt):
    # Keep only printable ascii letters, replace others with space.
    # This range of control characters is modified from
    # https://stackoverflow.com/a/39416125. We only have ascii characters so we
    # only need exclude non-printable ascii characters, not the full Unicode
    # range
    non_printable = re.compile(r'[\x00-\x1f\7f]')
    printable_txt = re.sub(non_printable, " ", txt)

    # Replace all punctuation with whitespace. The paper only states that they
    # removed punctuation, but doing it this way ensures that words separated by
    # a period or a parenthesis, for example, will be properly split.
    # I learned to use re.escape from https://stackoverflow.com/a/266162
    punct = string.punctuation
    punct_patt = re.compile('[{punct}]'.format(punct=re.escape(punct)))
    no_punct_txt = re.sub(punct_patt, " ", printable_txt)

    # split into bag of words using whitespace
    ws = re.compile(r'\s+')
    tokens = re.split(ws, no_punct_txt)


    # Remove numbers, except those that are in words with alphabetic characters.
    # This is to allow for the possible use of numbers and punctuation (periods,
    # underscores) in method, class names, etc. The paper was not clear as to
    # whether they removed all numbers or simply those tokens that consisted
    # only of numerals. I have taken the latter approach.
    digits = string.digits
    alpha = string.ascii_letters

    # First add only words without any numbers to the new list.
    alpha_only_tokens = [t for t in tokens
                         if len(set(t).intersection(set(digits))) == 0]

    # Add words that contain letters *and* numbers.
    alpha_only_tokens.extend([t for t in tokens
                              if len(set(t).intersection(set(alpha))) > 0
                              and len(set(t).intersection(set(digits))) > 0])

    # Remove stop words, that is, words which generally do not add much
    # meaning to a sentence like articles and conjunctions.
    sw = stopwords.words("english")
    tokens = [t for t in alpha_only_tokens if t not in sw]

    # Split compound words written in CamelCase into parts based on capital
    # letters, and augment our token list with those parts.
    for t in tokens:
        t_parts = split_camelcase(t)

        # Words not in CamelCase will have only one list element: the original
        # word. Otherwise, augment our token list with the individual parts.
        if len(t_parts) > 1:
            for t_part in t_parts:
                tokens.append(t_part)

    # Get the morphological stem of the word using the Porter stemmer from
    # ntlk. We might also consider using lemmatization rather than stemming.
    # This is more computationally intensive, and so can take considerably
    # longer, but it yields more accurate results. I have included commented
    # code using a lemmatizer. The stemmer converts most words to lowercase,
    # while the lemmatizer does not, so this is something we might need to
    # tweak if we decide to use a lemmatizer.
    stemmer = porter.PorterStemmer()
    base_tokens = [stemmer.stem(t) for t in tokens]

    # Using a lemmatizer
    # lemmatizer = WordNetLemmatizer()
    # base_tokens = [lemmatizer.lemmatize(t) for t in tokens]

    # Final check to make sure there are no empty strings
    tokens = [t for t in base_tokens if t]

    return tokens


def build_vocab(txt):
    # Create a processed list of unique words from a document
    doc_words = preprocess(txt)
    vocab = set(doc_words)
    return vocab


def count_vectorize(corpus, vocab=None):
    # Tokenize and vectorize text with simple count frequencies.
    vectorizer = CountVectorizer(analyzer=preprocess, vocabulary=vocab)

    count_vector = vectorizer.fit_transform(corpus)

    count_vector_arr = count_vector.toarray()

    return count_vector_arr


def tfidf_vectorize_query(query, corpus, vocab=None, return_cv=False):
    q_cv_array = count_vectorize([query], vocab=vocab)

    q_vector = q_cv_array[0]

    # Number of documents in corpus
    n = int(corpus.shape[0])

    # Number of terms in corpus
    tc = len(q_vector)

    # Initialize list that will hold file counts for each term with zeroes
    files_per_term = [0 for i in range(tc)]

    # calculate the number of documents in which each term appears
    for row in corpus:
        for col_num, cell in enumerate(row):
            if cell > 0:
                files_per_term[col_num] += 1

    # Initialize TD-IDF vector for query
    q_tfidf_list = [0.0 for j in range(tc)]

    # `a` is the smoothing term. See
    # https://nlp.stanford.edu/IR-book/html/htmledition/maximum-tf-normalization-1.html
    a = 0.5

    max_tf = 0
    for tf in q_vector:
        if tf > max_tf:
            max_tf = tf
    for i, tf in enumerate(q_vector):
        # We leave the weighted term frequency as 0 if a term does not appear
        # in the string.
        if tf > 0:
            # Calculate normalized term weight according to method used in
            # Ye et al.
            nf = a + (1.0 - a) * (tf/max_tf)

            # Calculate idf. Here we add one to the numerator and
            # denominator to prevent the possibility of division by zero. This
            # is not mentioned in the paper, but is the method of "smoothing"
            # used by the sklearn tfidf vectorizer.
            idf = math.log10((n + 1)/(files_per_term[i] + 1))

            # Calculate adjusted weight and store in TD-IDF vector array
            wf = nf * idf
            q_tfidf_list[i] = wf

    # Convert list to numpy array
    q_tfidf_arr = numpy.array(q_tfidf_list)

    if return_cv:
        return q_tfidf_arr, q_vector
    else:
        return q_tfidf_arr


def tfidf_vectorize_corpus(corpus, vocab=None, return_cv=False,
                           count_vector_array=None):
    if count_vector_array is None:
        cv_array = count_vectorize(corpus, vocab=vocab)
    else:
        cv_array = count_vector_array

    # Initialize number of documents in corpus
    n = 0

    # Number of terms in corpus
    tc = len(cv_array[0])

    # Initialize list that will hold file counts for each term with zeroes
    files_per_term = [0 for i in range(tc)]

    # calculate the number of document in corpus and number of files in which
    # each term appears
    for row in cv_array:
        n += 1
        for col_num, cell in enumerate(row):
            if cell > 0:
                files_per_term[col_num] += 1

    # Initialize TD-IDF array with zeroes
    tfidf_list = [[0.0 for j in range(tc)] for i in range(n)]

    # `a` is the smoothing term. See
    # https://nlp.stanford.edu/IR-book/html/htmledition/maximum-tf-normalization-1.html
    a = 0.5

    for i, row in enumerate(cv_array):
        max_tf = 0
        for tf in row:
            if tf > max_tf:
                max_tf = tf
        for j, tf in enumerate(row):
            # We leave the weighted term frequency as 0 if a term does not
            # appear in the string.
            if tf > 0:
                # Calculate normalized term weight according to method used in
                # Ye et al.
                nf = a + (1.0 - a) * (tf/max_tf)

                # Calculate idf. Here we add one to the numerator and
                # denominator to prevent the possibility of division by zero.
                # This is not mentioned in the paper, but is the method of
                # "smoothing" used by the sklearn tfidf vectorizer.
                idf = math.log10((n + 1)/(files_per_term[j] + 1))

                # Calculate adjusted weight and store in TD-IDF vector array
                wf = nf * idf
                tfidf_list[i][j] = wf


    # Convert array to numpy array
    tfidf_arr = numpy.array(tfidf_list)
    
    if return_cv:
        return tfidf_arr, cv_array
    else:
        return tfidf_arr

#mycorpus = ['A green tree', 'A red apple', 'A green apple', 'A red tree']
#query = 'Where is the apple tree'
#corpus_and_query = mycorpus.copy()
#corpus_and_query.append(query)
#corpus_and_query_str = '\n'.join(corpus_and_query)
#myvocab = sorted(list(build_vocab(corpus_and_query_str)))
#corpus_v = tfidf_vectorize_corpus(mycorpus, vocab=myvocab)
#query_v = tfidf_vectorize_query(query, corpus_v, vocab=myvocab)
#print(corpus_v)
#print(query_v)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Create vocabulary

In [0]:
# import pickle

# corpora = ''
# vocab = set()

# # Add each bug report
# for i, (bug_report, bug_report_summary) in enumerate(zip(brdf['description'],
#                                                          brdf['summary'])):
#     if i >= 6244:
#         print(f'Adding bug report at index {i}')
#         complete_report = '\n'.join([str(bug_report_summary), str(bug_report)])
#         corpora = '\n\n'.join([corpora, complete_report])

# chunk_vocab = build_vocab(corpora)
# vocab = vocab.union(chunk_vocab)
# corpora = ''
# print(list(vocab)[:100])
# print(len(vocab))

# unique_commits = []

# first_commit_index = int(brdf['id'].max()) - 1

# first_commit_src = \
#     extract_commit_corpus(brdf['previous_commit'][first_commit_index])

# # Add all files from first prefix commit
# print(f'Adding pre-fix commit at index {first_commit_index}.')
# corpora = '\n\n'.join([corpora, '\n'.join(first_commit_src)])

# prior_commit_index = first_commit_index
# commit_processed = 1

# # Add changed files from every subsequent commit
# for i in reversed(range(first_commit_index)):
#     # Thanks to https://stackoverflow.com/a/869914 for an efficient way of
#     # looping from high to low.
#     if brdf['previous_commit'][i] not in unique_commits:
#         print(f'Adding changed files from pre-fix commit at index {i}')
#         changed_files = \
#             eclipse_repo.git.diff('--name-only',
#                                   brdf['previous_commit'][prior_commit_index],
#                                   brdf['previous_commit'][i]).split('\n')
#         cur_commit_src = extract_commit_corpus(brdf['previous_commit'][i],
#                                                file_list=changed_files)
#         corpora = '\n\n'.join([corpora, '\n'.join(cur_commit_src)])
#         if commit_processed % 10 == 0:
#             chunk_vocab = build_vocab(corpora)
#             vocab = vocab.union(chunk_vocab)
#             corpora = ''
#             print(list(vocab)[:100])
#             print(len(vocab))
#         prior_commit_index = i
#         unique_commits.append(brdf['previous_commit'][i])
#         commit_processed += 1
#         if commit_processed > 250:
#             break

# with open('vocab.pickle', 'wb') as f:
#     pickle.dump(vocab, f)

Adding bug report at index 6244
Adding bug report at index 6245
Adding bug report at index 6246
Adding bug report at index 6247
Adding bug report at index 6248
Adding bug report at index 6249
Adding bug report at index 6250
Adding bug report at index 6251
Adding bug report at index 6252
Adding bug report at index 6253
Adding bug report at index 6254
Adding bug report at index 6255
Adding bug report at index 6256
Adding bug report at index 6257
Adding bug report at index 6258
Adding bug report at index 6259
Adding bug report at index 6260
Adding bug report at index 6261
Adding bug report at index 6262
Adding bug report at index 6263
Adding bug report at index 6264
Adding bug report at index 6265
Adding bug report at index 6266
Adding bug report at index 6267
Adding bug report at index 6268
Adding bug report at index 6269
Adding bug report at index 6270
Adding bug report at index 6271
Adding bug report at index 6272
Adding bug report at index 6273
Adding bug report at index 6274
Adding b

KeyboardInterrupt: ignored

'commit 4953e3e2db29253276e54381f4535c8c245bba20\nAuthor: Nick Edgar <nick>\nDate:   Thu Oct 11 16:53:17 2001 +0000\n\n    Text input for build 204\n\ncommit beda4e71ad1fb1ed72a5b44da73212cb3b290594\nAuthor: '

In [0]:
import pickle

with open('cs616/vocab1.pickle', 'rb') as f:
    vocab = pickle.load(f)

global_vocab = sorted(list(vocab))

len(global_vocab)

15824

In [0]:
def vectorize_prefix_commit(commit_index, vocab, prior_commit_data=None):
    # vectorizes a pre-fix commit with associated bug report
    commit_hash = brdf['previous_commit'][commit_index]

    if not prior_commit_data:
        commit_corpus = \
            extract_commit_corpus(commit_hash, return_names=True)
    else:
        # Learned about '--name-status' from
        # https://stackoverflow.com/a/30050755
        changed_files = \
            eclipse_repo.git.diff('--name-status', prior_commit_data[0],
                                  commit_hash).split('\n')

        deleted_files = [f.lstrip('D\t') for f in changed_files
                         if f.startswith('D\t') and f.endswith('.java') and not
                         str(pathlib.Path(f).name).startswith('.')]
        added_files = [f.lstrip('A\t') for f in changed_files
                       if f.startswith('A\t') and f.endswith('.java') and not
                       str(pathlib.Path(f).name).startswith('.')]
        modified_files = [f.lstrip('M\t') for f in changed_files
                          if f.startswith('M\t') and f.endswith('java') and not
                          str(pathlib.Path(f).name).startswith('.')]

        if added_files or deleted_files:
            # retrieve all, set prior data to none
            prior_commit_data = None
            commit_corpus = \
                extract_commit_corpus(commit_hash, return_names=True)
        else:
            # only retrieve modified
            commit_corpus = extract_commit_corpus(commit_hash,
                                                  file_list=modified_files,
                                                  return_names=True)

    corpus_filelist = commit_corpus[0::2]
    corpus_filecontents = commit_corpus[1::2]

    if prior_commit_data:
        corpus_filedict = \
            {fp:fc for (fp, fc) in zip(corpus_filelist, corpus_filecontents)}

        unmodified_files = list()
        unmodified_filecontents = list()
        unmodified_count_vectors = list()
        added_filecontents = list()
        modified_filecontents = list()

        for file_data in prior_commit_data[1]:
            if file_data[0] == 'bug_report':
                continue
            if file_data[0] not in deleted_files:
                if file_data[0] not in modified_files:
                    unmodified_files.append(file_data[0])
                    unmodified_count_vectors.append(file_data[2])
                    unmodified_filecontents.append(file_data[3])
                else:
                    modified_filecontents.append(corpus_filedict[file_data[0]])

        for filepath in added_files:
            added_filecontents.append(corpus_filedict[file_data[0]])

        new_count_vectors = count_vectorize([*modified_files, *added_files],
                                            vocab=vocab)
        count_vectors = numpy.array([*unmodified_count_vectors,
                                     *new_count_vectors])
        vect_corpus_arg = None
        corpus_filelist = [*unmodified_files, *modified_files, *added_files]
        corpus_filecontents = [*unmodified_filecontents,
                               *modified_filecontents, *added_filecontents]
    else:
        count_vectors = None
        vect_corpus_arg = corpus_filecontents

    # Vectorize pre-fix commit corpus
    print(f'Vectorizing pre-fix commit at index {commit_index}.')
    if prior_commit_data:
        print(len(corpus_filedict))
    tfidf_v, count_v = tfidf_vectorize_corpus(vect_corpus_arg,
                                              vocab=vocab,
                                              return_cv=True,
                                              count_vector_array=count_vectors)

    complete_query = '\n'.join([str(brdf['summary'][commit_index]),
                                str(brdf['description'][commit_index])])
    q_tfidf_v, q_count_v = tfidf_vectorize_query(complete_query,
                                                 corpus=tfidf_v,
                                                 vocab=vocab,
                                                 return_cv=True)

    corpus_data = list()
    corpus_data.append(('bug_report', q_tfidf_v, q_count_v, complete_query))

    for filepath, cv, tfidfv, cont in zip(corpus_filelist, tfidf_v, count_v,
                                          corpus_filecontents):
        corpus_data.append((filepath, cv, tfidfv, cont))

    return tuple((commit_hash, tuple(corpus_data)))


# first_commit_data = vectorize_prefix_commit(6494, vocab=global_vocab)
# second_commit_data = \
#     vectorize_prefix_commit(6493, vocab=global_vocab,
#                             prior_commit_data=first_commit_data)
# third_commit_data = \
#     vectorize_prefix_commit(6492, vocab=global_vocab,
#                             prior_commit_data=second_commit_data)

# content_differs_1_2 = 0
# cv_differs_1_2 = 0
# content_differs_2_3 = 0
# cv_differs_2_3 = 0

# for f1 in first_commit_data[1]:
#     for f2 in second_commit_data[1]:
#         if f1[0] == f2[0]:
#             if f1[0] != 'bug_report':
#                 if f1[3] != f2[3]:
#                     content_differs_1_2 += 1
#                 if not (f1[2] == f2[2]).all():
#                     cv_differs_1_2 += 1
#             break

# print(content_differs_1_2)
# print(cv_differs_1_2)

# for f1 in second_commit_data[1]:
#     for f2 in third_commit_data[1]:
#         if f1[0] == f2[0]:
#             if f1[0] != 'bug_report':
#                 if f1[3] != f2[3]:
#                     content_differs_2_3 += 1
#                 if not (f1[2] == f2[2]).all():
#                     cv_differs_2_3 += 1
#             break

# print(content_differs_2_3)
# print(cv_differs_2_3)

unique_commits = []

# first_commit_index = brdf.shape[0] - 1
# cur_commit_data = vectorize_prefix_commit(first_commit_index,
#                                           vocab=global_vocab)
# cur_commit_hash = brdf['previous_commit'][first_commit_index]

# with open(f'gdrive/My Drive/Colab Files/{first_commit_index}.pickle', 'wb') as f:
#     pickle.dump(cur_commit_data, f)

# unique_commits.append(cur_commit_hash)
previous_commit_data = None
commits_processed = 0

# Loop over the next oldest 249 commits
#for i in reversed(range(first_commit_index)):
for i in reversed(range(6362)):
    cur_commit_hash = brdf['previous_commit'][i]
    if cur_commit_hash not in unique_commits:
        # Thanks to https://stackoverflow.com/a/869914 for an efficient way of
        # looping from high to low.
        cur_commit_data = \
            vectorize_prefix_commit(i, vocab=global_vocab,
                                    prior_commit_data=previous_commit_data)

        with open(f'gdrive/My Drive/Colab Files/{i}.pickle', 'wb') as f:
            pickle.dump(cur_commit_data, f)

        unique_commits.append(cur_commit_hash)
        previous_commit_data = cur_commit_data
        commits_processed += 1

        if commits_processed == 117:
            break

Vectorizing pre-fix commit at index 6361.
Vectorizing pre-fix commit at index 6360.
7
Vectorizing pre-fix commit at index 6359.
3
Vectorizing pre-fix commit at index 6358.
1
Vectorizing pre-fix commit at index 6357.
12
Vectorizing pre-fix commit at index 6356.
1
Vectorizing pre-fix commit at index 6355.
1
Vectorizing pre-fix commit at index 6354.
4
Vectorizing pre-fix commit at index 6353.
1
Vectorizing pre-fix commit at index 6352.
4
Vectorizing pre-fix commit at index 6351.
2
Vectorizing pre-fix commit at index 6350.
2
Vectorizing pre-fix commit at index 6349.
6
Vectorizing pre-fix commit at index 6348.
3
Vectorizing pre-fix commit at index 6347.
4
Vectorizing pre-fix commit at index 6346.
1
Vectorizing pre-fix commit at index 6345.
Vectorizing pre-fix commit at index 6344.
2
Vectorizing pre-fix commit at index 6343.
1
Vectorizing pre-fix commit at index 6342.
8
Vectorizing pre-fix commit at index 6341.
1
Vectorizing pre-fix commit at index 6340.
10
Vectorizing pre-fix commit at inde